In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit, udf

### Data Reading

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@databrickproject1.dfs.core.windows.net/customers")

display(df)

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
df = df.withColumn('domains',split(col('email'),'@')[1])
display(df)

In [0]:
df.groupby('domains').agg(count("customer_id").alias("total_customers")).sort(desc("total_customers")).display()


In [0]:
df_gmail = df.filter(col('domains') == 'gmail.com')
display(df_gmail)

In [0]:
df = df.withColumn('full_name',concat(col('first_name'),lit(' '),col("last_name")))
df = df.drop('first_name','last_name')
display(df)

### Data Writing


In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databrickproject1.dfs.core.windows.net/customers")


In [0]:
%sql
create schema databricks_cata.silver

In [0]:
%sql 
describe schema  databricks_cata.silver

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.customers
using delta 
LOCATION "abfss://silver@databrickproject1.dfs.core.windows.net/customers"

In [0]:
%sql
select * from databricks_cata.silver.customers

In [0]:
# spark.sql("DROP SCHEMA IF EXISTS databricks_cata.silver CASCADE")
# spark.sql("DROP TABLE IF EXISTS databricks_cata.silver.customer_silver")